In [73]:
import numpy as np
from scipy.integrate import quad


def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15 # temperatura padrão 25 Celcius

class Comp: # Compostos : Classe que abriga as propriedades e funções
    def __init__(self, omega, Pc, Tb, Tc, NCParams, Ant1 = None, Ant2 = None, Ant3 = None): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant1 = Ant1
        self.Ant2 = Ant2
        self.Ant3 = Ant3
    
    def Ant(self, T): # faixas de antoine, Escolhe uma
        if T > self.Ant1[3] and T < self.Ant1[4]:
            return self.Ant1[:3]

        if T > self.Ant2[3] and T < self.Ant2[4]:
            return self.Ant2[:3]

        if T > self.Ant3[3] and T < self.Ant3[4]:
            return self.Ant3[:3]
        

    def Antoine(self, T): # NIST, T em K
        Ant = self.Ant(T)
        AntA = Ant[0]
        AntB = Ant[1]
        AntC = Ant[2]
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(self, P): # Eq antoine para temperatura
        Ant = self.Ant1
        T = (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]
        if self.Ant3 == None:
            return T
        Ant = self.Ant(T) # roda novamente para escolher a faixa correta de antoine
        return (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]

    def Estado(self, T, P): # Retorna o estado do composto
        if (T > self.Tc and P > self.Pc) or self.Ant1 == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if  P < self.Antoine(T):
                return "vapour"
            else:
                return "liquid"

    def Cp_ig(self, T): # Equação 142 V.Ness
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))
    
    def Cp_L(self, T): # Cp de Rowlison Bondi
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) )  + self.Cp_ig(T)
    
    def Hres(self, P, T): # Por Peng Robinson slide aula 2 integração
        P = toPa(P)
        k = 0.37464 + (0.154226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / self.Tc )
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / self.Tc)
        A = (a * P) / ((R ** 2) * (T ** 2.5))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * B ** 2 ) - 2 * B
        gamma = - (A * B) + (B ** 2) + (B ** 3)
        Z = np.roots([1, alpha, beta, gamma])
        Z = Z[np.isreal(Z)]
        Z = max(Z)
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self, T): # Por Riedel p147 V.Ness
        return R * T * ((1.092 * (np.log(self.Pc - 1.013)))/ (0.930 - (T / self.Tc))) # P em bar mesmo

    def Hmolar(self, P, T): # Calcula a entalpia molar
        match self.Estado(P, Tpad): # estado inicial
            case "gas" | "vapour":
                match self.Estado(P, T):
                    case "gas" | "vapour":
                        # print("caminho gas - gas")
                        H = quad(self.Cp_ig, Tpad, T)[0] + self.Hres(P, T)
                        # print(f"{H} J/mol")
                        return H
                    case "liquid":
                        # print("caminho gas - liquido")
                        Tvap = self.TAntoine(P)
                        HGas = quad(self.Cp_ig, Tpad, Tvap)[0] + self.Hres(P, Tvap)
                        Hv = self.Hvap(Tvap)
                        Hl = quad(self.Cp_L, Tvap, T)[0]
                        H = HGas - Hv + Hl # Menos Hvaporização já que é o processo inverso, condensação
                        # print(f"{H} J/mol")
                        return H
            case "liquid":
                match self.Estado(P, T):
                    case "gas" | "vapour":
                        # print("caminho liquido - gas")
                        Tvap = self.TAntoine(P)
                        Hl = quad(self.Cp_L, Tpad, Tvap)[0]
                        Hv = self.Hvap(Tvap)
                        HGas = quad(self.Cp_ig, Tvap, T)[0] + self.Hres(P, T)
                        H = Hl + Hv + HGas
                        # print(f"{H} J/mol\n")
                        return H
                    case "liquid":
                        # print("caminho liquido - liquido")
                        H = quad(self.Cp_L, Tpad, T)[0]
                        # print(f"{H} J/mol")
                        return H

H2 = Comp( -0.217, # Prausnitz
          12.93, #Bar
          20.27, #K
          32.98, #K
         [3.249,
         0.422e-3,
         0,
         0.083e5])

CO = Comp(
        0.045, # Prausnitz
        34.94, #Bar
        81.66, #K
        132.85, #K
        [ 3.376,
         0.557e-3,
         0,
         -0.031e5])

CO2 = Comp(
        0.225, # Prausnitz
        73.74, #Bar
        194.69, #K
        304.12, #K
        [ 5.457,
        1.045e-3,
        0,
        -1.157e5]) 

N2 = Comp(
        0.037,
        33.98,
        77.35,
        126.20,
        [3.280,
        0.593e-3,
        0,
        0.040e5])
    
CH3OH = Comp(
        0.565,
        80.97,
        337.69,
        512.64,
        [2.211, # A
        12.216e-3, # B
        -3.450e-6, # C
        0], # D
        [5.15853, 1569.613, -34.846, 353.5, 512.63], # A, B, C, D, Tmin, Tmax Ambrose 1975
        [5.20409, 1581.341, -33.50, 288.1, 356.83], # Ambrose 1970
        [5.31301, 1676.569, -21.728, 353, 483]) # Hirata 1967

H2O = Comp(
        0.344, # Prausnitz
        220.64, #Bar
        373.15, #K
        647.14, #K
        [3.470, # V.Ness - |
        1.450e-3,
        0,
        0.121e5],
        [3.55959, 643.748, -198.043, 373, 573], # A, B, C, D, Tmin, Tmax do NIST liu e lindsey
        [4.6543, 1435.264, -64.848, 255.9, 373] # Stull, 1947
        )


In [74]:
# Para descoberta da temperatura Corrente 3
C2 = [
    0,
    0,
    265.6704877,
    60.1569231,
    1163.056436,
    15.03923078,
]  # Vazões CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

T2 = toK(163)
P2 = 52
C12 = [
    6.24030213769374,
    2.41078129059267,
    213.428133248607,
    0,
    12251.7033530114,
    393.828789949421,
]  # Vazões CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

T12 = toK(22)
P12 = 52
# C-2
C_2 = (
    C2[2] * CO.Hmolar(P2, T2)
    + C2[3] * CO2.Hmolar(P2, T2)
    + C2[4] * H2.Hmolar(P2, T2)
    + C2[5] * N2.Hmolar(P2, T2)
) # Entalpia da corrente
# C-12
C_12 = (
    C12[0] * CH3OH.Hmolar(P12, T12)
    + C12[1] * H2O.Hmolar(P12, T12)
    + C12[2] * CO.Hmolar(P12, T12)
    + C12[3] * CO2.Hmolar(P12, T12)
    + C12[4] * H2.Hmolar(P12, T12)
    + C12[5] * N2.Hmolar(P12, T12)
) # Entalpia da corrente
# print(f"{C_12} kJ / mol H")
# C3
C3 = [
    6.24030213769363,
    2.41078129059269,
    479.098620992281,
    60.1569231043921,
    13414.759788997,
    408.868020725519,
]  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
P3 = 52  # Estimativa
C_3 = C_2 + C_12
T3 = 0
for T in np.arange(T12, T2, 0.01):
    C3ite = (
        C3[0] * CH3OH.Hmolar(P3, T)
        + C3[1] * H2O.Hmolar(P3, T)
        + C3[2] * CO.Hmolar(P3, T)
        + C3[3] * CO2.Hmolar(P3, T)
        + C3[4] * H2.Hmolar(P3, T)
        + C3[5] * N2.Hmolar(P3, T)
    )
    # print(f"Dif:{abs(C_3 - C3ite)}, T:{T - 273.15}")
    if C3ite < C_3 + 1e4 and C3ite > C_3 - 1e4:
        print(T - 273.15)
        T3 = T
        break

# C-3
C_3 = ( 
    C3[0] * CH3OH.Hmolar(P3, T3)
    + C3[1] * H2O.Hmolar(P3, T3)
    + C3[2] * CO.Hmolar(P3, T3)
    + C3[3] * CO2.Hmolar(P3, T3)
    + C3[4] * H2.Hmolar(P3, T3)
    + C3[5] * N2.Hmolar(P3, T3)
)


36.82999999998651


In [75]:
# TC-1
# Tubos C3 -> C4
# C-4 C3 = C4 vazão igual
T4 = toK(220)
P4 = 51
C_4 = (
    C3[0] * CH3OH.Hmolar(P4, T4)
    + C3[1] * H2O.Hmolar(P4, T4)
    + C3[2] * CO.Hmolar(P4, T4)
    + C3[3] * CO2.Hmolar(P4, T4)
    + C3[4] * H2.Hmolar(P4, T4)
    + C3[5] * N2.Hmolar(P4, T4)
)

QTC1 = C_3 - C_4 
print(QTC1)



846467304.1389084
